# 🚀 Electronics Detection Model Training - Google Colab

Train YOLOv8 model for electronics detection using **FREE GPU** on Google Colab.

**Estimated Training Time:** 2-3 hours (vs 24+ hours on CPU locally)

---

## 📋 Prerequisites

Before running this notebook:
1. ✅ Download your dataset (e.g., from [Roboflow Universe](https://universe.roboflow.com/sanctum/electronics-j0cxl))
2. ✅ Upload dataset to Google Drive (recommended) or upload directly to Colab
3. ✅ Enable GPU: **Runtime → Change runtime type → Hardware accelerator → GPU (T4)**

---

## Step 1: Check GPU Availability

Verify that GPU is enabled (should show Tesla T4 or similar).

In [ ]:
!nvidia-smi

## Step 2: Install Dependencies

Install Ultralytics (YOLOv8) and required packages.

In [ ]:
!pip install ultralytics

# Import libraries
from ultralytics import YOLO
import os
from pathlib import Path

print("✅ Dependencies installed successfully!")

## Step 3: Mount Google Drive (Optional but Recommended)

Mount your Google Drive to access the dataset and save results.

**Skip this cell if you'll upload the dataset directly to Colab.**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Drive mounted at /content/drive/MyDrive/")

## Step 4: Upload Dataset

### Option A: From Google Drive (Recommended)

If you uploaded your dataset to Google Drive, update the path below:

In [ ]:
# Update this path to match your dataset location in Google Drive
DATASET_PATH = "/content/drive/MyDrive/Electronics.v1i.yolov8"

# Verify dataset exists
if os.path.exists(DATASET_PATH):
    print(f"✅ Dataset found at: {DATASET_PATH}")
    print(f"📁 Contents: {os.listdir(DATASET_PATH)}")
else:
    print(f"❌ Dataset not found at: {DATASET_PATH}")
    print("Please update DATASET_PATH or use Option B to upload directly.")

### Option B: Upload Dataset Directly to Colab

**Alternative:** Upload your dataset.zip file using the file upload button below.

⚠️ **Note:** Files uploaded directly to Colab will be deleted when the session ends!

In [ ]:
# Uncomment and run this cell if uploading directly

# from google.colab import files
# import zipfile

# print("📤 Upload your dataset.zip file...")
# uploaded = files.upload()

# # Extract the uploaded zip
# for filename in uploaded.keys():
#     if filename.endswith('.zip'):
#         print(f"📦 Extracting {filename}...")
#         with zipfile.ZipFile(filename, 'r') as zip_ref:
#             zip_ref.extractall('/content/')
#         dataset_name = filename.replace('.zip', '')
#         DATASET_PATH = f"/content/{dataset_name}"
#         print(f"✅ Dataset extracted to: {DATASET_PATH}")
#         break

## Step 5: Configure Training Parameters & Auto-Resume

Adjust these settings based on your needs.

⚠️ **Free Tier Protection:** The notebook will automatically:
- Save checkpoints to Google Drive every N epochs
- Resume from the last checkpoint if training is interrupted
- Protect against session timeouts and disconnections

In [ ]:
# Training Configuration
CONFIG = {
    'model': 'yolov8n.pt',        # Base model (n=nano, s=small, m=medium)
    'custom_model': None,          # Path to your custom model for incremental training (e.g., '/content/drive/MyDrive/best.pt')
    'epochs': 100,                 # Number of training epochs
    'batch': 16,                   # Batch size (reduce if out of memory)
    'imgsz': 640,                  # Image size
    'device': 0,                   # GPU device (0 for first GPU)
    'patience': 20,                # Early stopping patience
    'save_period': 5,              # Save checkpoint every N epochs (reduced for free tier)
    'project': 'electronics_training',  # Output folder
    'name': 'colab_run',           # Experiment name
    
    # 🆕 AUTO-RESUME SETTINGS (for Colab free tier)
    'auto_resume': True,           # Automatically resume from last checkpoint
    'drive_backup': True,          # Backup checkpoints to Google Drive
    'drive_backup_path': '/content/drive/MyDrive/colab_training_backups',  # Google Drive backup location
}

# 📝 INCREMENTAL TRAINING INSTRUCTIONS:
# To train from a previously trained model instead of starting from scratch:
# 1. Upload your trained model (e.g., best.pt or last.pt) to Google Drive
# 2. Set 'custom_model' to the path of your model, e.g.:
#    CONFIG['custom_model'] = '/content/drive/MyDrive/electronics_training/colab_run/weights/best.pt'
# 3. Run the training - it will continue from your custom model!

# 🔄 AUTO-RESUME INSTRUCTIONS:
# If your Colab session times out or disconnects:
# 1. Simply re-run all cells - the notebook will automatically detect the last checkpoint
# 2. Training will resume from where it left off (no progress lost!)
# 3. Checkpoints are saved to Google Drive every 5 epochs

print("⚙️  Training Configuration:")
for key, value in CONFIG.items():
    print(f"   {key}: {value}")
    
if CONFIG['custom_model']:
    print("\n🔄 INCREMENTAL TRAINING MODE")
    print(f"   Base model: {CONFIG['custom_model']}")
    print("   Training will continue from this checkpoint!")
else:
    print(f"\n🆕 FRESH TRAINING MODE")
    print(f"   Starting from: {CONFIG['model']}")

if CONFIG['auto_resume']:
    print(f"\n✅ AUTO-RESUME ENABLED")
    print(f"   Checkpoints will be saved to: {CONFIG['drive_backup_path']}")
    print(f"   If session disconnects, just re-run and training will resume!")

## Step 5.5: (Optional) Keep Session Alive

⚠️ **For Free Tier Users:** Colab may disconnect after ~90 minutes of inactivity.

Run the cell below to add a keep-alive script (optional but recommended for long training sessions).


In [ ]:
# Keep-Alive Script (prevents Colab timeout)
# This simulates user activity to keep the session alive

from IPython.display import display, Javascript

display(Javascript('''
function KeepClicking(){
    console.log("Keep-Alive: Simulating activity...");
    document.querySelector("colab-connect-button").click();
}
setInterval(KeepClicking, 60000);  // Click every 60 seconds
'''))

print("✅ Keep-Alive script activated!")
print("   Session will stay active even during long training runs")
print("   (Simulates activity every 60 seconds)")
print("\n⚠️  Note: Even with keep-alive, Colab may disconnect after 12 hours")
print("   But don't worry - auto-resume will save your progress!")


## 📖 How Auto-Resume Works

### If Your Session Disconnects:

1. **Don't Panic!** Your progress is automatically saved to Google Drive every 5 epochs
2. **Re-run the notebook:** Just click "Runtime → Run all" 
3. **Automatic Resume:** The notebook will detect your last checkpoint and continue from there

### What Gets Saved:
- ✅ Model weights (`last.pt` and `best.pt`)
- ✅ Training progress (epoch number, optimizer state)
- ✅ All training results and metrics

### Backup Location:
Your checkpoints are saved to: `/content/drive/MyDrive/colab_training_backups/`

### Manual Resume (Advanced):
If you want to manually resume from a specific checkpoint:
```python
CONFIG['custom_model'] = '/content/drive/MyDrive/colab_training_backups/colab_run_last.pt'
```

---


In [ ]:
# Setup periodic backup during training (runs in background)
import threading
import time

def periodic_backup():
    """Background thread that backs up checkpoints every 10 minutes"""
    while training_active:
        time.sleep(600)  # Wait 10 minutes
        
        if not training_active:
            break
            
        try:
            results_dir = Path(CONFIG['project']) / CONFIG['name']
            last_model = results_dir / 'weights' / 'last.pt'
            
            if last_model.exists() and CONFIG['drive_backup']:
                backup_dir = CONFIG['drive_backup_path']
                backup_path = os.path.join(backup_dir, f"{CONFIG['name']}_last.pt")
                
                shutil.copy2(str(last_model), backup_path)
                print(f"\n💾 [Auto-Backup] Checkpoint saved to Google Drive at {time.strftime('%H:%M:%S')}")
        except Exception as e:
            print(f"\n⚠️  [Auto-Backup] Warning: {e}")

# Initialize backup thread
training_active = True

if CONFIG['drive_backup']:
    backup_thread = threading.Thread(target=periodic_backup, daemon=True)
    backup_thread.start()
    print("🔄 Background backup thread started (saves to Drive every 10 minutes)")
    print("   This provides extra protection against sudden disconnections\n")
else:
    print("⏭️  Background backup disabled (drive_backup=False)\n")


## Step 6: Start Training 🏋️

This will take **2-3 hours** on Colab's free T4 GPU.

⚠️ **Important:** Keep this tab open or Colab may disconnect!

In [ ]:
# ============================================================
# AUTO-RESUME LOGIC (for Colab free tier protection)
# ============================================================

import shutil

# Check for existing checkpoint in Google Drive backup
resume_checkpoint = None
if CONFIG['auto_resume'] and CONFIG['drive_backup']:
    backup_dir = CONFIG['drive_backup_path']
    checkpoint_path = os.path.join(backup_dir, f"{CONFIG['name']}_last.pt")
    
    if os.path.exists(checkpoint_path):
        resume_checkpoint = checkpoint_path
        print("🔍 CHECKPOINT DETECTED!")
        print(f"   Found: {checkpoint_path}")
        print(f"   Training will RESUME from this checkpoint\n")
    else:
        print("🔍 No previous checkpoint found")
        print(f"   Will start fresh training\n")

# Determine which model to use (priority: resume > custom > default)
if resume_checkpoint:
    # HIGHEST PRIORITY: Resume from interrupted training
    model_path = resume_checkpoint
    print(f"♻️  RESUMING TRAINING from checkpoint...")
    print(f"   Checkpoint: {model_path}")
    
elif CONFIG['custom_model'] is not None:
    # Use custom model for incremental training
    model_path = CONFIG['custom_model']
    
    # Verify custom model exists
    if not os.path.exists(model_path):
        raise FileNotFoundError(
            f"❌ Custom model not found at: {model_path}\n"
            f"Please check the path and ensure the model file exists in Google Drive."
        )
    
    print(f"🔄 INCREMENTAL TRAINING: Loading custom model...")
    print(f"   Model: {model_path}")
    
else:
    # Use default base model
    model_path = CONFIG['model']
    print(f"🆕 FRESH TRAINING: Loading base model...")
    print(f"   Model: {model_path}")

model = YOLO(model_path)
print("✅ Model loaded successfully!\n")

# Verify dataset path
data_yaml = os.path.join(DATASET_PATH, 'data.yaml')
if not os.path.exists(data_yaml):
    raise FileNotFoundError(f"❌ data.yaml not found at {data_yaml}")

print(f"🎯 Dataset: {DATASET_PATH}")
print(f"📊 Config: {data_yaml}")
print(f"\n🏋️  Starting training for {CONFIG['epochs']} epochs...\n")

# Create backup directory in Google Drive if it doesn't exist
if CONFIG['drive_backup']:
    os.makedirs(CONFIG['drive_backup_path'], exist_ok=True)
    print(f"💾 Checkpoint backup directory ready: {CONFIG['drive_backup_path']}\n")

# Train the model
results = model.train(
    data=data_yaml,
    epochs=CONFIG['epochs'],
    imgsz=CONFIG['imgsz'],
    batch=CONFIG['batch'],
    device=CONFIG['device'],
    project=CONFIG['project'],
    name=CONFIG['name'],
    
    # Performance settings
    patience=CONFIG['patience'],
    save=True,
    save_period=CONFIG['save_period'],
    
    # Optimization
    optimizer='auto',
    lr0=0.01,
    lrf=0.01,
    
    # Augmentation
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=0.0,
    translate=0.1,
    scale=0.5,
    shear=0.0,
    perspective=0.0,
    flipud=0.0,
    fliplr=0.5,
    mosaic=1.0,
    
    # Other
    workers=8,
    exist_ok=True,
    pretrained=True,
    verbose=True,
)

print("\n" + "="*60)
print("✅ Training Complete!")
print("="*60)

# ============================================================
# SAVE CHECKPOINT TO GOOGLE DRIVE (for auto-resume)
# ============================================================
if CONFIG['drive_backup']:
    print("\n💾 Backing up checkpoint to Google Drive...")
    
    results_dir = Path(CONFIG['project']) / CONFIG['name']
    last_model = results_dir / 'weights' / 'last.pt'
    best_model = results_dir / 'weights' / 'best.pt'
    
    backup_dir = CONFIG['drive_backup_path']
    
    if last_model.exists():
        backup_last = os.path.join(backup_dir, f"{CONFIG['name']}_last.pt")
        shutil.copy2(str(last_model), backup_last)
        print(f"   ✅ Saved: {backup_last}")
    
    if best_model.exists():
        backup_best = os.path.join(backup_dir, f"{CONFIG['name']}_best.pt")
        shutil.copy2(str(best_model), backup_best)
        print(f"   ✅ Saved: {backup_best}")
    
    print(f"\n🛡️  Your progress is safe in Google Drive!")
    print(f"   If this session disconnects, re-run the notebook to resume.\n")

## Step 7: Validate the Model

Run validation on the test set to check performance.

In [ ]:
# Get the path to the best model
results_dir = Path(CONFIG['project']) / CONFIG['name']
best_model_path = results_dir / 'weights' / 'best.pt'

print(f"🔍 Validating model: {best_model_path}\n")

# Load best model
best_model = YOLO(str(best_model_path))

# Validate
metrics = best_model.val()

# Display metrics
print("\n📈 Validation Metrics:")
print(f"   mAP50: {metrics.box.map50:.3f}")
print(f"   mAP50-95: {metrics.box.map:.3f}")
print(f"   Precision: {metrics.box.mp:.3f}")
print(f"   Recall: {metrics.box.mr:.3f}")

## Step 8: Test on Sample Images

Test the trained model on a few validation images.

In [ ]:
import glob
from IPython.display import Image, display

# Get sample images from validation set
val_images = glob.glob(os.path.join(DATASET_PATH, 'valid/images/*'))[:5]

print(f"🧪 Testing on {len(val_images)} sample images...\n")

for img_path in val_images:
    print(f"📸 Testing: {os.path.basename(img_path)}")
    
    # Run inference
    results = best_model(img_path)
    
    # Display results
    for result in results:
        boxes = result.boxes
        print(f"   Detected {len(boxes)} objects:")
        for box in boxes:
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            name = best_model.names[cls]
            print(f"      - {name}: {conf:.2%}")
    print()

## Step 9: View Training Results

Display training curves and results.

In [ ]:
from IPython.display import Image, display

results_dir = Path(CONFIG['project']) / CONFIG['name']

print("📊 Training Results:\n")

# Display results plot
results_plot = results_dir / 'results.png'
if results_plot.exists():
    print("📈 Training Curves:")
    display(Image(filename=str(results_plot)))

# Display confusion matrix
confusion_matrix = results_dir / 'confusion_matrix.png'
if confusion_matrix.exists():
    print("\n🎯 Confusion Matrix:")
    display(Image(filename=str(confusion_matrix)))

# Display labels
labels_plot = results_dir / 'labels.jpg'
if labels_plot.exists():
    print("\n🏷️  Dataset Labels Distribution:")
    display(Image(filename=str(labels_plot)))

## Step 10: Download Trained Model

Download the trained model to use in your local detector.

In [ ]:
from google.colab import files
import shutil

results_dir = Path(CONFIG['project']) / CONFIG['name']
best_model_path = results_dir / 'weights' / 'best.pt'
last_model_path = results_dir / 'weights' / 'last.pt'

print("📥 Preparing models for download...\n")

# Create a zip file with all results
output_zip = f"{CONFIG['name']}_trained_model.zip"
shutil.make_archive(output_zip.replace('.zip', ''), 'zip', str(results_dir))

print(f"✅ Created: {output_zip}")
print(f"📦 Size: {os.path.getsize(output_zip) / (1024*1024):.2f} MB\n")

# Download the zip
print("⬇️  Downloading...")
files.download(output_zip)

print("\n" + "="*60)
print("✅ Download Complete!")
print("="*60)
print("\n📝 Next Steps:")
print("1. Extract the zip file on your local machine")
print("2. Copy 'weights/best.pt' to your project folder")
print("3. Update electronics_detector.py:")
print("   self.model = YOLO('path/to/best.pt')")
print("4. Run your detector!")

## Step 11: (Optional) Save to Google Drive

Save the trained model and results to Google Drive for long-term storage.

In [ ]:
# Uncomment and run if you want to save to Google Drive

# import shutil

# results_dir = Path(CONFIG['project']) / CONFIG['name']
# drive_save_path = '/content/drive/MyDrive/electronics_training_results'

# print(f"💾 Copying results to Google Drive...")
# shutil.copytree(str(results_dir), drive_save_path, dirs_exist_ok=True)

# print(f"✅ Results saved to: {drive_save_path}")
# print("\nYour results are now permanently saved in Google Drive!")

---

## 🎉 Training Complete!

### Summary:
- ✅ Model trained on GPU (much faster than CPU)
- ✅ Best weights saved: `weights/best.pt`
- ✅ Results downloaded or saved to Google Drive

### Using Your Trained Model:

```python
# In electronics_detector.py, replace:
self.model = YOLO('yolov8n.pt')

# With:
self.model = YOLO('path/to/your/best.pt')
```

---

## 📚 Resources:
- [YOLOv8 Documentation](https://docs.ultralytics.com/)
- [Roboflow Universe Datasets](https://universe.roboflow.com/)
- [Project GitHub](https://github.com/GAM3RG33K/electronics_detector)

---

**Need Help?** Check the README.md or open an issue on GitHub.